<a href="https://colab.research.google.com/github/reynaldo-prado/reinaldoprad/blob/main/classificador_sentimentos_Reinaldo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar bibliotecas necessárias (roda só uma vez)
!pip install scikit-learn nltk pandas



In [2]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# Vamos pegar textos sobre dois assuntos diferentes
data = fetch_20newsgroups(subset='train', categories=['rec.sport.hockey', 'sci.space'])

# Criar uma tabela com os textos e rótulos (0 e 1)
df = pd.DataFrame({'text': data.data, 'label': data.target})

# Visualizar as 5 primeiras linhas
df.head()


,text,label
0,From: e8l6@jupiter.sun.csd.unb.ca (Rocket)\nSu...,0
1,From: umfu0009@ccu.umanitoba.ca (J. M. K. Fu)\...,0
2,From: Mark.Prado@p2.f349.n109.z1.permanet.org ...,1
3,From: igor@pravda.tse.su\nSubject: Who will br...,0
4,From: u1452@penelope.sdsc.edu (Jeff Bytof - SI...,1


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in stop_words and t not in string.punctuation]
    return ' '.join(tokens)

df['cleaned'] = df['text'].apply(clean_text)

df[['text', 'cleaned']].head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [4]:
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in stop_words and t not in string.punctuation]
    return ' '.join(tokens)

df['cleaned'] = df['text'].apply(clean_text)

df[['text', 'cleaned']].head()


,text,cleaned
0,From: e8l6@jupiter.sun.csd.unb.ca (Rocket)\nSu...,e8l6 jupiter.sun.csd.unb.ca rocket subject nhl...
1,From: umfu0009@ccu.umanitoba.ca (J. M. K. Fu)\...,umfu0009 ccu.umanitoba.ca j. m. k. fu subject ...
2,From: Mark.Prado@p2.f349.n109.z1.permanet.org ...,mark.prado p2.f349.n109.z1.permanet.org mark p...
3,From: igor@pravda.tse.su\nSubject: Who will br...,igor pravda.tse.su subject broadcast wc origin...
4,From: u1452@penelope.sdsc.edu (Jeff Bytof - SI...,u1452 penelope.sdsc.edu jeff bytof sio subject...


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Transformar os textos em números (vetores)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cleaned'])
y = df['label']

# Separar dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Treinar o modelo Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

# Testar o modelo e mostrar a acurácia
preds = model.predict(X_test)
print(f"Acurácia do modelo: {accuracy_score(y_test, preds) * 100:.2f}%")


Acurácia do modelo: 99.58%


In [7]:
def prever_sentimento(frase):
    frase_limpa = clean_text(frase)
    frase_vetorizada = vectorizer.transform([frase_limpa])
    resultado = model.predict(frase_vetorizada)
    return "Espaço (1)" if resultado[0] == 1 else "Esporte (0)"

# Teste com suas frases
print(prever_sentimento("NASA discovered a new planet near Jupiter"))
print(prever_sentimento("The hockey team played an amazing game yesterday"))
print(prever_sentimento("This new technology will change the world"))
print(prever_sentimento("The players were very skilled in the last match"))


Espaço (1)
Esporte (0)
Espaço (1)
Esporte (0)
